In [8]:
import pandas as pd
#import geopandas as gpd
import shapely
from shapely.geometry import Point, MultiPoint
#import fiona
#from pykml import parser
from shapely.ops import nearest_points

In [75]:
path = 'C:/Users/Malwina/Desktop/Data_Science/Bootcamp/Visualization/Bergfex_project_cont/coordinates.csv'
snow = pd.read_csv(path, delimiter = ';', decimal=',')
snow = snow.dropna()
# EPSG:4326/WGS84

path = 'C:/Users/Malwina/Desktop/Data_Science/Bootcamp/Visualization/Bergfex_project_cont/swiss_cities.csv'
points = pd.read_csv(path)

In [76]:
snow.head()

,location,height_in_m,snowlevel_in_cm,lat,lng
2,Aigle,381,0,46.319025,6.970566
3,Altdorf,449,0,46.880600,8.639400
4,Andermatt,2282,0,46.602389,8.611800
5,Appenzell,780,0,47.333300,9.416700
6,Beatenberg,1182,0,46.703541,7.809551


In [77]:
snow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48 entries, 2 to 76
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   location         48 non-null     object 
 1   height_in_m      48 non-null     int64  
 2   snowlevel_in_cm  48 non-null     int64  
 3   lat              48 non-null     float64
 4   lng              48 non-null     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 2.2+ KB


In [78]:
# we want to calculate the closest point in that df to another gps point
test_points = points.iloc[0:10, 0:4] # selecting a test set for the calculation
test_points

,Unnamed: 0,city,lat,lng
0,0,Zürich,47.3786,8.5400
1,1,Geneva,46.2000,6.1500
2,2,Basel,47.5606,7.5906
3,3,Lausanne,46.5333,6.6333
4,4,Bern,46.9480,7.4474
5,5,Winterthur,47.4992,8.7267
6,6,Lucerne,47.0523,8.3059
7,7,Sankt Gallen,47.4233,9.3772
8,8,Lugano,46.0103,8.9625
9,9,Biel/Bienne,47.1372,7.2472


In [79]:
# making a single multipoint object out of all the snow points (3 to test)
# making Points out of the snow points

# creating a grid out of all the snow points in the df snow
snow_point_list= []

for index, row in snow.iterrows():
    point = Point(row['lat'], row['lng'])
    snow_point_list.append(point)
    
len(snow_point_list) # 48 locations 

snow_grid = MultiPoint(snow_point_list)

In [84]:
# finding which snow point is nearest to the test point 
nearest_list = [] # list that contains tuples of nearest snow point in Point object format
near_lng = []  # contains the longitudes of the nearest snow point extracted from nearest shapely calculation
near_lat = []  # latitudes 

for index, row in test_points.iterrows():
    tested_point = Point(row['lat'], row['lng'])
    nearest = nearest_points(tested_point, snow_grid)
    nearest_list.append(nearest[1]) # the second item in the tuple is the nearest point

for i in range(len(nearest_list)):
    near_lng.append(nearest_list[i].y) 
    near_lat.append(nearest_list[i].x)

In [85]:
len(near_lng)

10

In [86]:
# adding the coordinated of the nearest snow point to each row of the hike df

gps_df = test_points

gps_df['near_lat'] = near_lat
gps_df['near_lng'] = near_lng




In [87]:
gps_df

,Unnamed: 0,city,lat,lng,near_lat,near_lng
0,0,Zürich,47.3786,8.5400,47.378600,8.540000
1,1,Geneva,46.2000,6.1500,46.200000,6.150000
2,2,Basel,47.5606,7.5906,47.537870,7.570988
3,3,Lausanne,46.5333,6.6333,46.535202,6.541900
4,4,Bern,46.9480,7.4474,46.998256,7.451340
5,5,Winterthur,47.4992,8.7267,47.378600,8.540000
6,6,Lucerne,47.0523,8.3059,47.085600,8.442100
7,7,Sankt Gallen,47.4233,9.3772,47.423300,9.377200
8,8,Lugano,46.0103,8.9625,46.171747,8.790342
9,9,Biel/Bienne,47.1372,7.2472,46.998256,7.451340


In [ ]:
# now we need to match those calculated nearest coordinates with the name of the snow_point with same coordinates